In [45]:
require 'nn'
require 'itorch'
require 'image'
require 'optim'
require 'xlua'
require 'paths'

require 'model.VanillaLSTM'
require 'util.DataLoader'
require 'util.BatchLoader'
require 'util.KFoldSlidingWindowDataLoader'
require 'util.LatentVariableDataLoader'
require 'model.MultiScaleLatentConvLSTM'
require 'model.ReconsLayer'

utils = require 'util.utils'

dtype = 'torch.FloatTensor'

In [46]:
opt = {}
-- Dataset options
opt.input_h5 = '../../openface_data/mohit_data.h5'
opt.train_seq_h5 = '../../openface_data/main_gest_by_file_1.h5'
opt.data_dir = '../../openface_data/face_gestures/dataseto_text'
opt.aug_gests_h5 = '../../openface_data/main_gest_by_file_aug_1_K_32.h5'
opt.batch_size = 200
opt.num_classes = 11
opt.num_classify = 5
opt.win_len = 10
opt.win_step = 5
opt.num_features = 28
-- Return the list of which batch inputs are in the current batch for validation
-- data
opt.val_batch_info = 1

opt.use_label_correction = 1

-- Model options
opt.init_from = './final_experiments/label_corr/exp1/checkpoint_26.t7'
opt.use_dense_conv_lstm = 0

-- Output options
--[[
cmd:option('-save', 'dense_step_5_cls_5')
cmd:option('-checkpoint_name', 'checkpoint')
cmd:option('-test_log', 'test.log')
cmd:option('-test_batch', 'test_batch.json')
cmd:option('-test_scores', 'test_scores.json')
cmd:option('-test_preds', 'test_preds.json')
]]

-- Backend options
opt.gpu = 1
opt.gpu_backend = 'cuda'

--[[
opt.test_log= opt.save..'/'..opt.test_log
opt.test_scores = opt.save .. '/' .. opt.test_scores
opt.test_preds = opt.save .. '/' .. opt.test_preds
]]

--[[
local trainLogger = optim.Logger(paths.concat(opt.save, opt.train_log))
if opt.use_dense_conv_lstm == 1 then
  trainLogger:setNames{'train_err', 'train_loss',}
else
  trainLogger:setNames{'train_err', 'train_loss', 'l_inf_w1_16',
    'l_inf_w1_32', 'l_inf_w1_64'}
end
]]

In [47]:
if opt.gpu == 1 then
  require 'cutorch'
  require 'cunn'
  require 'cudnn'
  cutorch.setDevice(1)
  dtype = 'torch.CudaTensor'
  print(string.format('Running with CUDA on GPU 1'))
else
  print('Running in CPU mode')
end

-- Confusion matrix
local classes = {}
for i=1,opt.num_classify do table.insert(classes, i) end
-- Confusion matrix for data
conf = optim.ConfusionMatrix(classes)

opt_clone = torch.deserialize(torch.serialize(opt))

print('Will load model '..opt.init_from)
-- Load model
model = torch.load(opt_clone.init_from)

print('Did load model')
print(model.train_conf)
print(model.conf)
model = model.model
--print(model)

Running with CUDA on GPU 1	
Will load model ./final_experiments/label_corr/exp1/checkpoint_26.t7	


Did load model	
ConfusionMatrix:
[[   13308       0       3       2       7]   99.910% 	[class: 1]
 [       0   13309       3       2       6]   99.917% 	[class: 2]
 [       2       3   13307       2       6]   99.902% 	[class: 3]
 [       3       1       3   13311       2]   99.932% 	[class: 4]
 [       4       4       5       2   13305]]  99.887% 	[class: 5]
 + average row correct: 99.909909963608% 
 + average rowUcol correct (VOC measure): 99.820001125336% 
 + global correct: 99.90990990991%	
ConfusionMatrix:
[[   22634    7738     459    1739     906]   67.613% 	[class: 1]
 [     824    7777     122     451     127]   83.615% 	[class: 2]
 [     152      84     551     448      40]   43.216% 	[class: 3]
 [     524     214     139    3030      45]   76.670% 	[class: 4]
 [     249     275      88      82     652]]  48.440% 	[class: 5]
 + average row correct: 63.910565972328% 
 + average rowUcol correct (VOC measure): 41.347966194153% 
 + global correct: 70.200607902736%	


In [48]:
-- Get the reconstruction layer from the model we need to 
-- inspect its weights

reconsLayer = model.net:get(10)

function is_identity_matrix(m, n)
    if torch.sum(m) ~= n then return false end
    for i=1,n do if m[i][i] ~= 1 then return false end end
    return true
end
    
function is_diagonally_dominant(m, low, high)
    -- Check if all the diagonal values lie between low, high
    -- If they do its a diagonally dominant matrix
    for i=1,m:size(1) do
        local row_sum = torch.sum(m[{{i},{}}]) - m[i][i]
        local col_sum = torch.sum(m[{{},{i}}]) - m[i][i]
        if ((m[i][i] < low or m[i][i] > high) and 
            (row_sum > 0.2 or col_sum > 0.2))
            then return false end
    end
    return true
end
    
changed_weights, changed_weights_idx = {}, {}
actual_changed_weights, actual_changed_weights_idx = {}, {}
local W = reconsLayer.weight
for i=1,W:size(1), 5 do
    local curr_w = W[{{i,i+4},{}}]:clone()
    curr_w = curr_w:view(5,5)
    if not is_identity_matrix(curr_w, 5) then 
        table.insert(changed_weights, curr_w)
        table.insert(changed_weights_idx, i)
        if not is_diagonally_dominant(curr_w, 0.8, 1.2) then
            table.insert(actual_changed_weights, curr_w)
            table.insert(actual_changed_weights_idx, i)
        end
    end
end
    
local save_base_dir = paths.dirname(opt.init_from)
local save_file_name = paths.basename(opt.init_from, 't7')
save_file_name = save_file_name..'_recons_weights.t7'
save_file_name = paths.concat(save_base_dir, save_file_name)
local checkpoint = {
    changed_weights=changed_weights, 
    changed_weights_idx=changed_weights_idx,    
    actual_changed_weights=actual_changed_weights,
    actual_changed_weights_idx=actual_changed_weights_idx,
}
torch.save(save_file_name, checkpoint)
print('Did save changed weights as '..save_file_name)

Did save changed weights as /home/mohit/Projects/HumanGestureRecognition/conv_lstm/torch/final_experiments/label_corr/exp1/checkpoint_26_recons_weights.t7	


In [49]:
-- Display some random changed weights
print('Total changed weights: '..#changed_weights)
print('Total actual changed weights: '..#actual_changed_weights)
num_random = 10
print('Reconstruction matrix')
for i=1,num_random do
    local idx = torch.random(#changed_weights)
    print("Changed weight at idx: "..changed_weights_idx[idx])
    print(changed_weights[idx])
end

print('Non diagonal dominant reconstruction matrix')
for i=1,num_random do
    local idx = torch.random(#actual_changed_weights)
    print("Changed weight at idx: "..actual_changed_weights_idx[idx])
    print(actual_changed_weights[idx])
end



Total changed weights: 30429	
Total actual changed weights: 0	
Reconstruction matrix	
Changed weight at idx: 825731	
 0.9759 -0.0192  0.0216  0.0227  0.0258
 0.0234  1.0163 -0.0196 -0.0212 -0.0241
-0.0023 -0.0014  1.0014  0.0016  0.0016
-0.0025 -0.0015  0.0016  1.0016  0.0017
-0.0021 -0.0014  0.0014  0.0015  1.0015
[torch.FloatTensor of size 5x5]

Changed weight at idx: 433026	
 0.9812 -0.0183  0.0202  0.0216  0.0235
 0.0186  1.0181 -0.0201 -0.0214 -0.0233
-0.0050 -0.0052  1.0053  0.0057  0.0058
-0.0041 -0.0041  0.0041  1.0047  0.0050
-0.0033 -0.0035  0.0036  0.0043  1.0043
[torch.FloatTensor of size 5x5]

Changed weight at idx: 230021	
 1.0179  0.0097 -0.0093 -0.0200 -0.0218
-0.0188  0.9851  0.0145  0.0207  0.0239
-0.0076  0.0025  0.9970  0.0071  0.0079
-0.0045  0.0008 -0.0005  1.0067  0.0052
-0.0057 -0.0025  0.0026  0.0090  1.0061
[torch.FloatTensor of size 5x5]

Changed weight at idx: 314901	
 1.0224  0.0184 -0.0251 -0.0199 -0.0296
-0.0263  0.9781  0.0286  0.0233  0.0324
-0.0025  0.

bad argument #1 to '?' (upper bound must be strictly positive at /home/mohit/Projects/torch/pkg/torch/build/TensorMath.c:6486)
stack traceback:
	[C]: at 0x7f3b6f3cefe0
	[C]: in function 'random'
	[string "-- Display some random changed weights..."]:14: in main chunk
	[C]: in function 'xpcall'
	...hit/Projects/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...hit/Projects/torch/install/share/lua/5.1/itorch/main.lua:174>
	...hit/Projects/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	.../Projects/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	.../Projects/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	.../Projects/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...hit/Projects/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00405d50: 